In [1]:
import subprocess
import json
import os
from constants import *
from web3 import Web3
from bit import *
from dotenv import load_dotenv

from web3.middleware import geth_poa_middleware
from eth_account import Account

from pathlib import Path
from getpass import getpass

from bit import wif_to_key, PrivateKeyTestnet
from bit.network import NetworkAPI

load_dotenv()

True

In [2]:
mnemonic = os.getenv('Mnemonic')
mnemonic

'devote easily begin inmate aim hidden donkey mention swallow budget october shell crystal collect ceiling'

In [3]:
w3 = Web3(Web3.HTTPProvider("http://127.0.0.1:8545"))
w3.middleware_onion.inject(geth_poa_middleware, layer=0)
from web3.gas_strategies.time_based import medium_gas_price_strategy
w3.eth.setGasPriceStrategy(medium_gas_price_strategy)

In [4]:
def derive_wallets(Coin, Mnemonic, Numderive):
    command = f'php hd-wallet-derive\hd-wallet-derive.php -g --mnemonic="devote easily begin inmate aim hidden donkey mention swallow budget october shell crystal collect ceiling" --numderive=5 --coin=eth --cols=path,address,privkey,pubkey --format=json'
    print(command)
    p = subprocess.Popen(command, stdout=subprocess.PIPE, shell=True)
    output, err = p.communicate()
    p_status = p.wait()
    keys = json.loads(output)

    return keys
    

In [6]:
derive_wallets(ETH, mnemonic, 5)

php hd-wallet-derive\hd-wallet-derive.php -g --mnemonic="devote easily begin inmate aim hidden donkey mention swallow budget october shell crystal collect ceiling" --numderive=5 --coin=eth --cols=path,address,privkey,pubkey --format=json


[{'path': "m/44'/60'/0'/0/0",
  'address': '0xe5D5c87896ab758af680c67E745641d049754E97',
  'privkey': '0x51c4b01bb574f1a3da35678f44e9bc30419c6d2ce2fd959c2dca4f70c1866eaf',
  'pubkey': '023ef65209836479a6323e60e84e7917ebce0b163e990365bb9fb8f684fd6530dc'},
 {'path': "m/44'/60'/0'/0/1",
  'address': '0x6a22D1955e6bf5774B06a7CD08eECA0b60fF30B5',
  'privkey': '0x184d7504e39c97f0c2fdf039db44624dec97a8dd3e8df303bdb40c8b4e460d8a',
  'pubkey': '032ca087c22fc642cabe01d2423ef49f17c82a0fd78ac2ecca90d7c7b319c0a82d'},
 {'path': "m/44'/60'/0'/0/2",
  'address': '0xBE6b5B1FE8A215E77f8f26c7eF55D1A7b9D7A21C',
  'privkey': '0xa52d5040d92fb5cf72b81d450aa1391d0adf0bc6c79d6c2b8cc638bcc9871ee9',
  'pubkey': '024050277412b7f7f55aa272fe586c9f9253abe0799518ab6542b46529ec375f9d'},
 {'path': "m/44'/60'/0'/0/3",
  'address': '0x5eEF5C82f70b70ee89Cc472a75486F4789C70390',
  'privkey': '0x25779ba5a6d57eb4eaf466f8975ae232ea7bcd25ca5a79d5e5bb49c00e221674',
  'pubkey': '03a75d939ed092c837a834f9b030c25ae7d1d99fcc0e4cefea

In [7]:
coins = {ETH:derive_wallets(ETH, mnemonic, 5), BTCTEST:derive_wallets(BTCTEST, mnemonic, 5)}

php hd-wallet-derive\hd-wallet-derive.php -g --mnemonic="devote easily begin inmate aim hidden donkey mention swallow budget october shell crystal collect ceiling" --numderive=5 --coin=eth --cols=path,address,privkey,pubkey --format=json
php hd-wallet-derive\hd-wallet-derive.php -g --mnemonic="devote easily begin inmate aim hidden donkey mention swallow budget october shell crystal collect ceiling" --numderive=5 --coin=eth --cols=path,address,privkey,pubkey --format=json


In [8]:
def priv_key_to_account(coin, priv_key):
    if coin == ETH:
        return Account.privateKeyToAccount(priv_key)
    if coin == BTCTEST:
        return PrivateKeyTestnet(priv_key)

In [14]:
ETH_key = priv_key_to_account(ETH, "ca01aedcc1ae01d7a16f1168e48068c39b9b18f6b0493c5bb861a6d9d48d70ed")

# Create Transaction Function
def create_tx(coin, account, recipient, amount):
    if coin == ETH:
        gasEstimate = w3.eth.estimateGas(
            {"from": account.address, "to": recipient, "value": amount}
        )
        trx_data = {
            "to": recipient,
            "from": account.address,
            "value": amount,
            "gasPrice": w3.eth.gasPrice,
            "gas": gasEstimate,
            "nonce": w3.eth.getTransactionCount(account.address)
        }
        return trx_data

    if coin == BTCTEST:
        return PrivateKeyTestnet.prepare_transaction(account.address, [(recipient, amount, BTC)])

In [15]:
# Create Send Transaction Function
def send_tx(coin, account, recipient, amount):
    tx = create_tx(coin, account, recipient, amount)
    signed_tx = account.sign_transaction(tx)
    if coin == ETH:
        result = w3.eth.sendRawTransaction(signed_tx.rawTransaction)
        return result.hex()
    elif coin == BTCTEST:
        return NetworkAPI.broadcast_tx_testnet(signed_tx)

In [16]:
send_tx(ETH, ETH_key, "0x3fCeA50b1b212F55985577f01E282784D1332CEb", 2)

'0x460e753a5e23a6df8684275e421b578ceb9cf0f3ce643eeeb851a329dcbd0ec5'

In [17]:
w3.eth.get_balance(ETH_key.address)

1000961889139999999998

In [18]:
print(ETH_key.address)

0xF8037da56b3Cf2026F69fAe26adc51f67638C306
